In [36]:
import pandas as pd
import numpy as np
import re

In [99]:
offers_df = pd.read_csv("../data/offers.csv")

In [20]:
offers_df.head(5)

,id,title,description,url,district,price,rent,pets,furnished,rooms,area,elevator,floor,buildtype
0,1019999649,Wynajmę dwupokojowe mieszkanie o wysokim stand...,Wynajmę 2-pokojowe mieszkanie- o wysokim stand...,https://www.olx.pl/d/oferta/wynajme-dwupokojow...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,980054310,Wynajme kawlerke 25 m przy ul Wiktoryn,Wynajmę nową kawalerkę przy kolejce WKD Aleje ...,https://www.olx.pl/d/oferta/wynajme-kawlerke-2...,Włochy,2 800 zł,450 zł,Tak,Tak,Kawalerka,25 m²,Tak,NaN,NaN
2,1019467904,"Jasne mieszkanie na Starych Bielanach, Schroeg...",Do wynajęcia mieszkanie na Bielanach przy ul. ...,https://www.olx.pl/d/oferta/jasne-mieszkanie-n...,Bielany,3 000 zł,890 zł,Tak,Tak,2 pokoje,36 m²,Nie,NaN,NaN
3,1019772480,Mieszkanie kawalerka Warszawa Wola,Posiadam do wynajęcia kawalerkę w dzielnicy Wo...,https://www.olx.pl/d/oferta/mieszkanie-kawaler...,Wola,2 500 zł,350 zł,Tak,Tak,Kawalerka,27 m²,Tak,NaN,NaN
4,1018935783,"3- pokojowe mieszkanie, jasne i przestronne z ...","ENGLISH BELOW-- Nowoczesne, przestronne 79 m² ...",https://www.olx.pl/d/oferta/3-pokojowe-mieszka...,Mokotów,5 500 zł,1 100 zł,Nie,Tak,3 pokoje,79 m²,Tak,NaN,NaN


In [21]:
offers_df.columns

Index(['id', 'title', 'description', 'url', 'district', 'price', 'rent',
       'pets', 'furnished', 'rooms', 'area', 'elevator', 'floor', 'buildtype'],
      dtype='object')

In [100]:
offers_df.drop(columns=['url'], inplace=True)
offers_df.columns


Index(['id', 'title', 'description', 'district', 'price', 'rent', 'pets',
       'furnished', 'rooms', 'area', 'elevator', 'floor', 'buildtype'],
      dtype='object')

In [31]:
offers_df.head(3)

,id,title,description,district,price,rent,pets,furnished,rooms,area,elevator,floor,buildtype
0,1019999649,Wynajmę dwupokojowe mieszkanie o wysokim stand...,Wynajmę 2-pokojowe mieszkanie- o wysokim stand...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,980054310,Wynajme kawlerke 25 m przy ul Wiktoryn,Wynajmę nową kawalerkę przy kolejce WKD Aleje ...,Włochy,2 800 zł,450 zł,Tak,Tak,Kawalerka,25 m²,Tak,NaN,NaN
2,1019467904,"Jasne mieszkanie na Starych Bielanach, Schroeg...",Do wynajęcia mieszkanie na Bielanach przy ul. ...,Bielany,3 000 zł,890 zł,Tak,Tak,2 pokoje,36 m²,Nie,NaN,NaN


In [27]:
len(offers_df)

9452

In [41]:
offers_df.dtypes

id              int64
title          object
description    object
district       object
price          object
rent           object
pets           object
furnished      object
rooms          object
area           object
elevator       object
floor          object
buildtype      object
dtype: object

price / rent → numeric

In [101]:
def to_number(x):
    if pd.isna(x):
        return np.nan
    x = str(x)
    x = x.replace("zł", "").replace(" ", "").replace(",", ".")
    return pd.to_numeric(x)

offers_df["price"] = offers_df["price"].apply(to_number)
offers_df["rent"]  = offers_df["rent"].apply(to_number)

area → numeric

In [102]:
def extract_area(x):
    if pd.isna(x):
        return np.nan
    match = re.search(r"(\d+[\.,]?\d*)", str(x))
    return float(match.group(1).replace(",", ".")) if match else np.nan

offers_df["area"] = offers_df["area"].apply(extract_area)

rooms → numeric


In [103]:
def parse_rooms(x):
    if pd.isna(x):
        return np.nan
    
    x = str(x).lower()
    if "kawalerka" in x:
        return 1
    match = re.search(r"(\d+)", x)
    return int(match.group(1)) if match else np.nan

offers_df["rooms"] = offers_df["rooms"].apply(parse_rooms).astype("Int64")

floor → numeric

In [104]:
def parse_floor(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip().lower()
    if x == "parter":
        return 0
    return int(x) if x.isdigit() else np.nan

offers_df["floor"] = offers_df["floor"].apply(parse_floor).astype("Int64")


boolean columns

In [105]:
def yes_no(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip().lower()
    if x == "tak":
        return True
    if x == "nie":
        return False
    return np.nan

offers_df["pets"] = offers_df["pets"].apply(yes_no).astype("boolean")
offers_df["furnished"] = offers_df["furnished"].apply(yes_no).astype("boolean")
offers_df["elevator"]  = offers_df["elevator"].apply(yes_no).astype("boolean")

In [106]:
def near_subway(text):
    if pd.isna(text):
        return False
    text = str(text).lower()
    keywords = ["metro", "subway", "stacja metra", "metra", "blisko metra", "blisko stacji metra"] 
    return any(kw in text for kw in keywords)

offers_df["subway"] = offers_df["description"].apply(near_subway).astype("boolean") | offers_df["title"].apply(near_subway).astype("boolean")

In [95]:
offers_df["subway"].value_counts()

subway
True     5174
False    4278
Name: count, dtype: Int64

In [107]:
offers_df.tail()

,id,title,description,district,price,rent,pets,furnished,rooms,area,elevator,floor,buildtype,subway
9447,1031158533,"Studio w stylu boho, cowork, rabaty.","Studio w stylu boho, cowork, rabaty. ENGLI...",Praga-Północ,2810.0,690.0,<NA>,True,1,26.0,<NA>,6,Apartamentowiec,True
9448,1036622909,Mieszkanie w centrum - Krucza 19,"Instagramowe mieszkanie w centrum — Krucza 19,...",Śródmieście,5500.0,200.0,<NA>,True,2,40.0,<NA>,5,Apartamentowiec,False
9449,1034085938,Przestronna 2-pokojowa kawalerka z widokiem,Na wynajem przestronna i widna kawalerka o pow...,Śródmieście,2500.0,519.0,<NA>,True,2,39.4,<NA>,<NA>,Kamienica,True
9450,1037039835,"Klimatyczne mieszkanie, centrum Starówka wszęd...",Stylowe mieszkanie na Starówce – wszystko w za...,Śródmieście,3500.0,860.0,<NA>,True,2,35.5,<NA>,1,Kamienica,False
9451,1034338944,Mieszkanie wynajem grzybowska Śródmieście,"Super Nowoczesne, sloneczne, ciche mieszkanie ...",Śródmieście,3000.0,NaN,<NA>,True,2,28.0,<NA>,7,Blok,False


In [78]:
sorted(offers_df["district"].dropna().unique())

['Bemowo',
 'Białołęka',
 'Bielany',
 'Mokotów',
 'Ochota',
 'Praga-Południe',
 'Praga-Północ',
 'Rembertów',
 'Targówek',
 'Ursus',
 'Ursynów',
 'Wawer',
 'Wesoła',
 'Wilanów',
 'Wola',
 'Włochy',
 'Śródmieście',
 'Żoliborz']

In [82]:
sorted(offers_df["buildtype"].dropna().unique())

['Apartamentowiec',
 'Blok',
 'Dom wolnostojący',
 'Kamienica',
 'Pozostałe',
 'Szeregowiec']

In [108]:
offers_df["district"] = offers_df["district"].astype("category")
offers_df["buildtype"] = offers_df["buildtype"].astype("category")

In [109]:
offers_df.dtypes

id                int64
title            object
description      object
district       category
price           float64
rent            float64
pets            boolean
furnished       boolean
rooms             Int64
area            float64
elevator        boolean
floor             Int64
buildtype      category
subway          boolean
dtype: object

handling missing values

In [110]:
offers_df.isnull().sum()

id                0
title             0
description       0
district        109
price             1
rent           1387
pets           5501
furnished         1
rooms             2
area              1
elevator       5172
floor          9278
buildtype      9359
subway            0
dtype: int64

In [ ]:
offers_df.dropna(subset=["district", "price", "furnished", "rooms", "area"], inplace=True)

# 2️⃣ Fill missing rent with 0
offers_df["rent"] = offers_df["rent"].fillna(0)

# 3️⃣ Fill missing elevator values with True
offers_df["elevator"] = offers_df["elevator"].fillna(True)

# 4️⃣ Drop columns floor and buildtype
offers_df.drop(columns=["floor", "buildtype"], inplace=True)

# Check result
offers_df.info()